# Configuración del entorno

En este notebook, se configurará rápidamente el entorno de Databricks para su uso con Azure Data Lake Storage Gen2 (ADLS Gen2).

## Objetivos

1. Inicializar el sistema de archivos de ADLS Gen2 usando la opción de configuración `fs.azure.createRemoteFileSystemDuringInitialization` para permitir que el sistema de archivos sea creado cuando se hace referencia por primera vez.
2. Montar el almacenamiento de Azure Data Lake Gen2 (ADLS Gen2) usando el acceso OAuth.

### Creación del punto de montaje

Para la creación del punto de montaje es necesario declarar las siguientes variables:
* ` adlsGen2AccountName`:  Nombre de la cuenta de almacenamiento.
* ` fileSystemName`: Nombre del sistema de archivos (o contenedor)
* ` scopeName`: Nombre del secret scope creado en databricks
* ` keyName`: Nombre del secreto creado en el Azure Key Vault

> *Nota 1*: Como buena práctica se usa un Azure Key Vault para llamar los secretos usando la función ` dbutils.secrets.get`.

Luego de la definición de las variables de conexión se hace el punto de montado a Databricks usando la función `dbutils.fs.mount`.

> *Nota 2*: La configuración `fs.azure.createRemoteFileSystemDuringInitialization` no interfiere en la creación del punto de montado si el sistema de archivos ya existe.

In [0]:
# Declaraciones variables.
adlsGen2AccountName = "analiticastorage"
fileSystemName      = "donaciones"
scopeName           = ""
keyName             = ""
accountKey          = "RB0bRK08z1Dcpy7adopANQyTJ7bXZ6XPSi2IxE6DDEkuKFysTCuPFsCUI1HxLv1xGUTvclCi85IeFAggb73XoQ=="

# Declaración de conexión
uri     = f"fs.azure.account.key.{adlsGen2AccountName}.blob.core.windows.net"
wasbUri = f"wasbs://{fileSystemName}@{adlsGen2AccountName}.blob.core.windows.net"

# Unión de los ajustes de configuración necesarios para el acceso OAuth a la cuenta ADLS Gen2.
configs  = {uri:accountKey}

# Antes de poder acceder al contenedor en su cuenta ADLS Gen2, debe inicializar un sistema de archivos. Para ello, se utilizara la opción de configuración `fs.azure.createRemoteFileSystemDuringInitialization` para permitir que el sistema de archivos se cree durante la operación de montaje. Se fijará este valor en `true` inmediatamente antes de acceder al sistema de archivos ADLS Gen2, y luego se volvera a `false` siguiendo el comando.

spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")

# Creación del punto de montaje del sistema de archivos en ADLS Gen2.
dbutils.fs.mount(
  source = wasbUri,
  mount_point = f"/mnt/{fileSystemName}",
  extra_configs = configs)

# Deshabilitar la creación del sistema de archivos durante la inicialización.
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-2963109138042931> in <module> 
 18 
 19 # Creación del punto de montaje del sistema de archivos en ADLS Gen2. 
 ---> 20 dbutils.fs.mount(
 21 source = wasbUri , 
 22 mount_point = f"/mnt/{fileSystemName}" , 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 387 exc . __context__ = None 
 388 exc . __cause__ = None 
 --> 389 raise exc
 390 
 391 return f_with_exception_handling

 ExecutionError : An error occurred while calling o362.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/donaciones; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/donaciones
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:756)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:776)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/donaciones
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:461)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$1(MetadataManager.scala:826)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:607)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:815)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:469)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:114)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:54)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:53)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:53)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$4.applyOrElse(DbfsServerBackend.scala:349)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$4.applyOrElse(DbfsServerBackend.scala:308)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:120)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:147)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:147)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:102)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:541)
	at com.databricks.logging.UsageLoggin

In [0]:
#dbutils.fs.unmount(mount_point = f"/mnt/{fileSystemName}")

### Prueba del punto de montaje

Para verificar rápidamente el acceso a el sistema de archivos ADLS Gen2, se ejecuta la celda de abajo.
%md

En la salida del comando `dbutils.fs.mounts()`, deberá ver el punto de montaje en la lista. Se verá algo así como: `mountPoint='/mnt/<Nombre del sistema de archivos>', source='abfss://<Nombre del sistema de archivos>@<Nombre de cuenta adls-gen2>.dfs.core.windows.net/'`.

In [0]:
display(
  dbutils.fs.ls(f"/mnt/{fileSystemName}/")
)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2963109138042934> in <module> 
 1 display(
 ----> 2 dbutils . fs . ls ( f"/mnt/{fileSystemName}/" ) 
 3 )

 NameError : name 'fileSystemName' is not defined